<a href="https://colab.research.google.com/github/keesterbrugge/rl_intro_sutton/blob/main/ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example 4.2: Jack’s Car Rental Jack manages two locations for a nationwide car
rental company. Each day, some number of customers arrive at each location to rent cars.
If Jack has a car available, he rents it out and is credited 10 by the national company.
If he is out of cars at that location, then the business is lost. Cars become available for
renting the day after they are returned. To help ensure that cars are available where
they are needed, Jack can move them between the two locations overnight, at a cost of
$2 per car moved. We assume that the number of cars requested and returned at each
location are Poisson random variables, meaning that the probability that the number is n is  n e  , where   is the expected number. Suppose   is 3 and 4 for rental requests at n!
the first and second locations and 3 and 2 for returns. To simplify the problem slightly, we assume that there can be no more than 20 cars at each location (any additional cars are returned to the nationwide company, and thus disappear from the problem) and a
maximum of five cars can be moved from one location to the other in one night. We take the discount rate to be   = 0.9 and formulate this as a continuing finite MDP, where the time steps are days, the state is the number of cars at each location at the end of the day, and the actions are the net numbers of cars moved between the two locations overnight. Figure 4.2 shows the sequence of policies found by policy iteration starting from the policy that never moves any cars.

In [118]:
from scipy.stats import poisson
import numpy as np
import numba as nb
# poisson([1,3,4,5]).pmf(k=np.array([4, 5]).T)
# poisson.pmf(mu=np.array([3,4]),k=np.array([4, 5]), size=(2,2))

In [ ]:




def get_new_V(s, gamma = .9):
  """
  state update consists of
  - stochastic amount of new cars at each location (3 and 2)
  - stochastic amount of car rental requests (3 and 4)
  - deterministic change in amount of cars based on choice to move from 1 loc to another
  
  Reward signal:
  - deterministically based on action (- 2 dollar per moved car)
  - deterministic based on amount of rentals (10 per rented car)
  """

  # don't have to loop over actions, cause policy is deterministc
  # loop over all state transitions 


<ipython-input-29-fab789a404ec>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([(2,3), 2])


array([(2, 3), 2], dtype=object)

In [ ]:
r(s_prime, pi(s), s) 
allowable_a = clip(a)
morning_s = s + allowable_a
a_allowed

In [44]:
tuple(np.array([3,3]))

(3, 3)

In [45]:
np.array((2,3))

array([2, 3])

In [39]:
s = (2,3)
(s[0] + (-2, 3)

(2, 3, -2, 3)

In [ ]:
def get_r(s,a):
  reward_action = - 2 * abs(a)
  s_after_action = np.clip(np.array(s) + np.array([-a, a]) a_min=0, a_max=20)
  mor

In [131]:

def trunc_poisson(mu, k, k_max):  
    if k < k_max: 
      p = poisson.pmf(mu=3, k=k)
    elif (k == k_max):
      p = poisson.sf(mu=3, k = k-1)
    else:
      p = 0.0
    return p

def get_p_s_prime_1loc(cars_start, rental_req_lambda, car_returns_lambda):

  max_cars = 20
  p_s_prime_1loc = np.zeros(max_cars+1)

  for cars_rented_out in np.arange(cars_start+1):
    p_cars_rented_out = trunc_poisson(mu=rental_req_lambda, k=cars_rented_out, k_max=cars_start)

    cars_after_renting_out = cars_start - cars_rented_out
    max_new_cars = max_cars - cars_after_renting_out

    for cars_arrived in np.arange(max_new_cars+1):

      p_cars_arrived = trunc_poisson(mu=car_returns_lambda, k=cars_arrived, k_max=max_new_cars)
      cars_end = cars_after_renting_out + cars_arrived
      p_state_transition = p_cars_rented_out * p_cars_arrived
      p_s_prime_1loc[cars_end] += p_state_transition

  return p_s_prime_1loc

In [82]:
def get_prob_per_new_state(s, rental_request_lambdas, car_returns_lambdas):
  return np.outer(*[get_p_s_prime_1loc(s[i], rental_request_lambdas[i], car_returns_lambdas[i]) for i in [0, 1]])

get_prob_per_new_state((10,10), (3,4), (3,2)).sum()

1.0000000000000004

In [73]:
gamma = 0.9
max_cars=20
V = np.zeros((max_cars+1,max_cars+1))
(gamma*V*p_s_prime).sum()

0.0

In [83]:

def get_expected_R(s, rental_request_lambdas):
  expected_cars_rented_out = 0
  for cars_start, rental_req_lambda in zip(s, rental_request_lambdas):
    expected_cars_rented_out += np.array(
        [cars_rented_out * trunc_poisson(mu=rental_req_lambda, 
                                        k=cars_rented_out, 
                                        k_max=cars_start) 
        for cars_rented_out in np.arange(cars_start+1)]).sum()

    
  return 10 * expected_cars_rented_out

get_expected_R(s=(3,3), rental_request_lambdas=(4,3))

46.55749154067674

In [ ]:
def get_r(s,a):
  np.array(for cars_start in s:


In [26]:

def (s, policy, V)

s = (10,10)
action = policy[s]
reward_action = - 2 * abs(a)
s_after_action = np.clip(np.array(s) + np.array([-a, a]) a_min=0, a_max=20)

cars_end_of_day = 7
action = -3
cars_start = cars_end_of_day + action

max_cars = 20

# P_S_prime = 3

# p_cars_rented_out = np.zeros(shape=20)
# p_cars_arrived = np.zeros(shape=20)


p_cars_rented_out[cars_start] = poisson.sf(mu=3, k = cars_start-1)
weighted_state_value_new_state = 0


    
  
for loc, rental_req_lambda, car_returns_lambda in zip([0,1], [3,4], [3,2]):
state_value = abs(action) * -2

for cars_rented_out in np.arange(cars_start+1):
  p_cars_rented_out = trunc_poisson(mu=3, k=cars_rented_out, k_max=cars_start)

  cars_after_renting_out = cars_start - cars_rented_out
  max_new_cars = max_cars - cars_after_renting_out

  for cars_arrived in np.arange(max_new_cars):

    p_cars_arrived = trunc_poisson(mu=3, k=cars_arrived, k_max=max_new_cars)

    # p_cars_arrived = poisson.pmf(mu=3, k=cars_arrived)
    # if (cars_arrived == max_new_cars):
    #   p_cars_arrived = poisson.sf(mu=3, k = cars_start-1)

      cars_end = cars_after_renting_out + cars_arrived

      p_state_transition = p_cars_rented_out * p_cars_arrived
      
      state_value += p_state_transition * (cars_rented_out * 10 + V[]
      

p_cars_rented_out





array([0.04978707, 0.14936121, 0.22404181, 0.22404181, 0.35276811,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [38]:
p_state_1loc_after_rental = 5 + trunc_poisson(mu, k, k_max):  


8

In [63]:


state_rental_req_lambda, car_returns_lambda, cars_start = 4, 4, 3
get_p_s_prime_1loc(4, 3, 3)
# trunc_poisson(mu=state_rental_req_lambda, k=cars_rented_out, k_max=cars_start)








array([1.75632901e-02, 6.38442550e-02, 1.23652345e-01, 1.70128948e-01,
       1.84453089e-01, 1.64305854e-01, 1.22634048e-01, 7.77544078e-02,
       4.24161839e-02, 2.01566182e-02, 8.44243938e-03, 3.15013070e-03,
       1.05719694e-03, 3.21829750e-04, 8.95314730e-05, 2.29114072e-05,
       5.42455129e-06, 1.19434804e-06, 2.45653449e-07, 4.73910054e-08,
       1.03656463e-08])

In [138]:
def get_s_morning_start(s, action):
  return(tuple(np.clip(np.array(s) + np.array([-action, action]), a_min=0, a_max=20).astype('int'))
)

In [137]:
%%time

# 1 init
V = np.random.uniform(size=(21,21))
A = np.arange(-5,6)
policy=np.zeros_like(V)
print(f"{V[:5,:5]=} \n\n {policy[:5,:5]=}")



rental_request_lambdas=(3,4)
car_return_lambdas = (3,2)
gamma = 0.9

# 2 
delta_theshold = 3

delta = delta_theshold + 1

# while delta >= delta_theshold:
for _ in range(5):
  delta = 0
  # for s in [(19,1)]:
  for s in np.random.permutation(list(np.ndindex(V.shape))):
    s = tuple(s)
    v = V[s]
    # expected_R = get_expected_R(s, rental_request_lambdas)
    action = policy[s]
    s_after_action = get_s_morning_start(s, action)
    actual_action = s_after_action[1] - s[1]
    action_R = - 2 * abs(actual_action)

    # s_after_action = tuple(np.clip(np.array(s) + np.array([-action, action]), a_min=0, a_max=20).astype('int'))
    # print(f"{s_after_action=}")

    expected_R = get_expected_R(s_after_action, rental_request_lambdas)
    P_new_state = get_prob_per_new_state(s_after_action, 
                                         rental_request_lambdas, 
                                         car_return_lambdas)
    # print(f"sanity: {P_new_state.sum()=}")
    expected_V_new_state = (V*P_new_state).sum()
    V[s] =  expected_R + action_R + gamma * expected_V_new_state

    delta = max(delta, abs(v-V[s]))

  print(f"sanity: {delta=}")


print(f"{V[:8,:8]=} \n\n {policy[:8,:8]=}")

# 3
# while delta >= delta_theshold:
for _ in range(5):
  policy_stable = True
  for s in np.random.permutation(list(np.ndindex(V.shape))):
    s = tuple(s)
    old_action = policy[s]
    s_after_action = get_s_morning_start(s, action)
    old_actual_action = s_after_action[1] - s[1]
    old_action_R = - 2 * abs(old_actual_action)
    best_action = old_actual_action
    for alt_action in A:
      alt_action_state_in_morning = get_s_morning_start(s, alt_action)
      

print(f"{V[:8,:8]=} \n\n {policy[:8,:8]=}")


V[:5,:5]=array([[0.1598447 , 0.46059418, 0.63788565, 0.56290277, 0.54804419],
       [0.64449234, 0.61233503, 0.82464332, 0.02681707, 0.95836638],
       [0.52064236, 0.91209836, 0.04263048, 0.44812278, 0.57409305],
       [0.50972473, 0.74782228, 0.71968424, 0.25296903, 0.00490525],
       [0.08635293, 0.08315032, 0.54779756, 0.68301723, 0.83563651]]) 

 policy[:5,:5]=array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])
sanity: delta=142.48830622195268
sanity: delta=145.4205264343295
sanity: delta=122.0816366875533
sanity: delta=100.87365107943734
sanity: delta=83.90495289667723
V[:8,:8]=array([[252.96940967, 243.46460658, 262.20226389, 271.16920835,
        273.53511086, 291.60804667, 262.69534038, 272.92811283],
       [227.3555574 , 250.46935542, 273.13195748, 290.02966653,
        282.32508023, 296.53777047, 302.78366465, 297.46295461],
       [246.35952221, 251.25259829, 287.28363872, 282

In [108]:
a = np.ndindex(V.shape)
a